In [16]:
# latest update
# April 26, 2017
# Changes done by: Jimmy Vivas
# Added code description

# This code was developed by Jimmy Vivas

# The code will take all CSv files from a specific directory and process them at the same Time
# The code will get replace strings in the values for numeric data
#   It will assign an integer value for each distinct string for each column (sensor)
#   The integer value will be reset it for each sensor
#   The code does not evaluare wether or not the sensor should stay. THe analyst will make this decision
#   Using Previse or any other tool
# The code will split the csv file if the result is more than 1MM rows

In [17]:
from os import listdir
import pandas as pd
import math
import numpy as np

In [18]:
# mac Path
#CSVPath1 =  '/Users/jvivas/Documents/XMT Baytwon Sensor Data' \
#            '/Sensor Data/Testing'

CSVPath1 = '/Users/jvivas/Documents/Aspen/GASCO/Sensor Data/To Be Processed'

# Windows Path
# CSVPath1 = 'C:/Users/jvivas/Dropbox/Mtell Customer Projects/XOM BayTown RHC MEA Tower Foaming/Sensor Data/' \
#           'To be Processed'

# CSVPath1 = 'C:/Mtell/Projects/XOM Baytown POC/Sensor Data/ToBeFormatted'

In [19]:
CSVFileList = []
NanPHDTagList = []
CSVFileListAll = listdir(CSVPath1)

m = len(CSVFileListAll)

for i in range(m):
    fileNameStr = CSVFileListAll[i]
    fileStr = fileNameStr.split('.')[0]
    fileExt = fileNameStr.split('.')[1]
    if fileExt == "csv":
        CSVFileList.append(fileNameStr)

n = len(CSVFileList)

# Uncomment following cell for debugging purposes

In [20]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# np.set_printoptions(threshold=np.nan)

In [21]:
def GetFileSize(file):
    import os
    csvFileStatInfo = os.stat(file)
    csvFileSizeGB = csvFileStatInfo.st_size/1000000000
    return(csvFileSizeGB)    

In [22]:
def RenameColumn(df2):
# Ranane Date and time column to DATETIME
    new_cols = ['DATETIME']
    df2.rename(columns=dict(zip(df2.columns[[0]], new_cols)),inplace=True)
    return

In [23]:
def ApplyDateFormat(df2):

    # duplicate df2 to apply date and time format 
    df2 = df2.copy()    
    # Change new datetime column to datetime format
    df2['DATETIME'] = pd.to_datetime(df2['DATETIME'])    
    # Change datetime column format to look like 01/31/2015 0:00:00
    df2['DATETIME'] = df2['DATETIME'].dt.strftime('%m/%d/%Y %H:%M:%S')
    return

In [24]:
def SplitCSVFile_GetStrings(CSVFileWithPath, CSVFile):
    chunksize = 25000

    i = 0
    j = 1
    
    SensorStringListAllChunks = []
    StringListForAllSensors = []
    StringListForEachChunk = []
        
    print ('Loading ' + CSVFile + ' file')
    for df in pd.read_csv(CSVFileWithPath, chunksize=chunksize, iterator=True, low_memory=False):
        # df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) 
        df.index += j
        i+=1
        j = df.index[-1] + 1
        h = 0
        
        # Rename date and time column
        print ('Renaming date and time column')
        RenameColumn(df)

        # Apply Date and time format to dataframe
        print ('Applying date and time format')
        ApplyDateFormat(df)

        # Code to do Transposing
        # Create two dataframes df1 only with tags and descriptions. df2 tag with values
        if i == 1:
            df1 = df[0:1]  # FIRST ROW
            # Indexing dataframe df1
            df1 = df1.set_index('DATETIME')
        else:
            if i > 1:
                df1 = df1
                
        df2 = df[1:len(df)]  # SECOND TO LAST ROW

        # Indexing dataframe df2
        df2 = df2.set_index('DATETIME')
        
        # Export CSV chunk for each loop
        print ('Exporting chunk' + str(i))
        df.to_csv(CSVFileWithPath.replace('.csv', '') + '_chunk_ ' + str(i) + '.csv', index=True)
        
        print ('Getting the list for chunk' + str(i))
        StringListForEachChunk = ExtractStrings(df1, df2)
        
        print ('These are all the strings found in the chunk ' + str(i) + ':' + str(StringListForEachChunk))
        
        SensorStringListAllChunks.extend(StringListForEachChunk)

    SensorStringListAllChunks = list(set(SensorStringListAllChunks))
    return(SensorStringListAllChunks)

In [25]:
def ExtractStrings(df2_1, df2_2):
        
    print ('Converting df to numeric')
    # Convert columns to numbers those that has string wil be converted to numpy null = NaN
    df2_with_nan = df2_2.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    # print (df2_2.head())
    # print (df2_with_nan.head())
    # Ge the list of sensors with Strings
    df3 = pd.DataFrame(df2_with_nan.isnull().any(axis=0))
    df3 = df3.reset_index()
    SensorsWithStrings = df3[df3[0]==True]['index'].tolist()
    
    # Replace nan with 'null' and create new df
    df2_with_null = df2_with_nan.fillna(value='null')
    
    # Here we filter the columns that are objects and create a dataframe with those columns
    colsObject = df2_2.select_dtypes(include=['object']).columns
    TotalColumns = df2_2.columns
    
    print ('Total columns with text: ' + str(TotalColumns))
    print ('Sensors with text in their Values: ' + str(SensorsWithStrings))

    AllSensorsStringList = []
    
    if len(SensorsWithStrings) == 0:
        print ('No strings in this dataframe')
    else:
        # print (str(len(SensorsWithStrings)) + ' columns contain strings out of ' + str(len(TotalColumns)) + ' columns')
        # if (len(SensorsWithStrings)/len(TotalColumns)) > 0.5:
            # print ('this process will take some time depending of the size of the file and the PC resources')

        # Dropping DATETIME index to merge df1 and df2
        df2_1 = df2_1.reset_index(drop=False)
        df2_2 = df2_2.reset_index(drop=False)
        
        # Variable initialization
        j = 0

        StringListForCurrentSensor = []

        # Create dictionary from dataframe columns (sensors) that have strings only
        SensorDictionary = {}.fromkeys(SensorsWithStrings, [])
        
        # Loop to go thru each column and convert the characters string to numbers    
        for j in range(len(SensorsWithStrings)):
            # from IPython.core.debugger import Tracer
            # Tracer()() #this one triggers the debugger

            # iterate thru each column in the dataframe
            # for j in range(len(list(SensorDictionary))):
            # update sensor name for each column
            Sensor = list(SensorDictionary)[j]

            print ('Processing Sensor: ' + Sensor)
            # Clear List of String for each Sensor
            SensorStringResult = []

            # Get rows that have null in the actual sensor column
            result = df2_with_nan[df2_with_nan[Sensor].isnull()][Sensor]

            # Convert list of nulls to dataframe and reset the datetime index
            result_df = pd.DataFrame(result)
            result_df = result_df.reset_index()
            
            # print ('printing result_df')
            # print (result_df.head())
            # print ('printing df2_2')
            # print (df2_2.head())
            # debugging
            # print (result_df)
            # debugging

            # Filtering rows with String for each sensor
            SensorStringResult = df2_2[df2_2['DATETIME'].isin(result_df.loc[:,"DATETIME"].values.tolist())][Sensor]

            # Adding Strings to Sensor in Sensor Dictionary
            # I think that when due to a bug the dictionary become too big it stops growing...
            # SensorDictionary[Sensor] = list(set(SensorStringResult))
            StringListForCurrentSensor = list(set(SensorStringResult))
            
            # Debugging
            # print (list(set(SensorStringResult)))
            # Debugging
            
            # print ('Sensor: ' + Sensor + ' Processed')

            # Adding String for Sensor to General String List
            AllSensorsStringList.extend(StringListForCurrentSensor)

            # Removing Duplicate Strings
            AllSensorsStringList = list(set(AllSensorsStringList))
            
            print ('These are all the strings found in Sensor ' + Sensor + ': ' + str(StringListForCurrentSensor))
    return (AllSensorsStringList)

In [26]:
def SetIndex(df2):
        # Code to do Transposing
    # Create two dataframes df1 only with tags and descriptions. df2 tag with values
    df2_1 = df2[0:1]  # FIRST ROW
    df2_2 = df2[1:len(df2)]  # SECOND TO LAST ROW
    
    # Indexing dataframe df1
    df2_1 = df2_1.set_index('DATETIME')

    # Indexing dataframe df2
    df2_2 = df2_2.set_index('DATETIME')
    return(df2_1, df2_2)

In [27]:
def LoadCSV(FileAndPath, file):
    print ('Loading ' + file)
    df2 = pd.read_csv(FileAndPath, low_memory=False)
    return(df2)

In [28]:
def FormatToPrevise(df2_1, df2_2):
    # Dropping DATETime index to merge df1 and df2
    df2_1 = df2_1.reset_index(drop=False)
    df2_2 = df2_2.reset_index(drop=False)

    # Converting Historian files to VTQ format (DATETime, TAGNAME, DESCRIPTION, VALUE)
    mdf = pd.merge(pd.melt(df2_1, id_vars=['DATETIME'], var_name='TAGNAME',
                           value_name='DESCRIPTION')[['TAGNAME', 'DESCRIPTION']],
                   pd.melt(df2_2, id_vars=['DATETIME'], var_name='TAGNAME',
                           value_name='VALUE'),
                   on=['TAGNAME'])

    # Sort columns by VTQ format
    mdf = mdf[['DATETIME', 'TAGNAME', 'DESCRIPTION', 'VALUE']]
    
    return (mdf)

In [48]:
range(5,)

range(0, 5)

In [49]:
def SplitPreviseFormatCSVFile(mdf):
    # Exporting PHD Tag CSV file
    i = 0
    rows = 1000000
    totalRows = len(mdf)
    loops = math.ceil(totalRows/rows) + 1

    if totalRows > 1000000:
        for j in range(loops): #need to round this
            j = j + 1
            print (str(j))
            a = (rows*j) - rows
            if totalRows <= rows:
                b = totalRows
                print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' Historian File')
                print("")
                mdf[a:b].to_csv(CSVPath1 + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted.csv', index=False)
            else:
                if (rows*j) >= totalRows:
                    b = totalRows
                    print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' chunk' + str(j) + ' Historian File')
                    print("")
                    mdf[a:b].to_csv(CSVPath1 + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted_chunk' + str(j) + '.csv', index=False)
                else:
                    b = (rows*j) - 1
                    print('Exporting ' + str(CSVFileList[i].replace('.csv', '')) + ' chunk' + str(j) + ' Historian File')
                    print("")
                    mdf[a:b].to_csv(CSVPath1 + '/' + str(CSVFileList[i].replace('.csv', '')) + '_Formatted_chunk' + str(j) + '.csv', index=False)
    else:
        mdf.to_csv(CSVPath1 + '/' + str(CSVFileList[0].replace('.csv', '')) + '_Formatted' + '.csv', index=False)
    return

# This is the Main Code to execute

In [30]:
CSVFile = CSVFileList[0]
CSVFileWithPath = CSVPath1 + "/" + CSVFileList[0]

csvFileSizeGB = GetFileSize(CSVFileWithPath)

if csvFileSizeGB > 1:
    StringListForAllSensors = SplitCSVFile_GetStrings(CSVFileWithPath, CSVFile)
    print ('These are all the strings found in the file: ' + str(StringListForAllSensors))
else:
    
    df2 = LoadCSV(CSVFileWithPath, CSVFile)
    
    # Rename date and time column
    print ('Renaming date and time column')
    RenameColumn(df2)
    # Apply Date and time format to dataframe
    print ('Applying date and time format')
    ApplyDateFormat(df2)
    
    # Apply Index and create two dataframes
    df2_1, df2_2 = SetIndex(df2)
    
    print ('Extracting strings from csv file')
    StringListForAllSensors = ExtractStrings(df2_1, df2_2)
    
    print ('These are all the strings found in the csv file: ' + str(StringListForAllSensors))

# Convert all strings to a Dictionary
StringListDict = {}.fromkeys(StringListForAllSensors, 'null')

Loading Tag_Data_2014_NoDups.csv
Renaming date and time column
Applying date and time format
Extracting strings from csv file
Converting df to numeric
Total columns with text: Index(['BUH02EDPV1052', 'BUH02EM0102A', 'BUH02EM0102B', 'BUH02EM0102C',
       'BUH02EM0102D', 'BUH02EM0102E', 'BUH02EM0102F', 'BUH02PDI1201',
       'BUH02PDI1277', 'BUH02PDI1302',
       ...
       'BUH02FI1087', 'BUH02TIC1000', 'BUH02TIC1001', 'BUH02FYX1050B',
       'BUH02HIC1050OP', 'BUH02FIC1247', 'BUH02TIC1006', 'BUH02FYX1087',
       'BUH02HIC2087OP', 'BUH02PI1440'],
      dtype='object', length=158)
Sensors with text in their Values: ['BUH02EDPV1052', 'BUH02EM0102A', 'BUH02EM0102B', 'BUH02EM0102C', 'BUH02EM0102D', 'BUH02EM0102E', 'BUH02EM0102F', 'BUH02PDI1201', 'BUH02PDI1277', 'BUH02PDI1302', 'BUH02PI1002', 'BUH02PI1004', 'BUH02PI1050', 'BUH02PI1265', 'BUH02PI1266', 'BUH02PI1267', 'BUH02PI1332A2', 'BUH02PI1334', 'BUH02PI1372', 'BUH02PI1392', 'BUH02PI1395', 'BUH02PIC1415', 'BUH02PIC1416', 'BUH02SI1315A', 

In [31]:
# CSVFile = CSVFileList[0]
# CSVFileWithPath = CSVPath1 + "/" + CSVFileList[0]

# StringListForAllSensors = SplitCSVFile_GetStrings(CSVFileWithPath, CSVFile)

# Run following line to see the list of Strings
## Copy the result and paste it in the following line of code

In [32]:
StringListDict

{'Bad': 'null',
 'Bad Input': 'null',
 'CLOSE': 'null',
 'ON': 'null',
 'OPEN': 'null',
 'Off': 'null',
 'null': 'null'}

# Now we can replace string with null or a number
## I have assigned null to all strings by default all strings are replaced with null. If the strings need to be replaced with a number please replace the null with your desired number
###Example
###{'String 1': 'null',
    'String 2': 1,
    'String 3': 0}

### The replace code is still Work in Progress for files bigger than 1 GB
#### What to do
##### 1. Since the code does not load the whole CSV but splits it. The code needs to open each splitted file and do the replace
##### 2. Once text is replaced the CSV file will be updated and load the next
##### 3. Once all csv file are processed We need to convert them to Previse Format

In [33]:
# Replacing known test to null
df2_2.replace(
{'Bad': 'null',
 'Bad Input': 'null',
 'CLOSE': 0,
 'ON': 1,
 'OPEN': 1,
 'Off': 0,
 'null': 'null'}
, inplace=True, regex=True)

# Now we convert the data to linear format Previse

## -----------------------------------------------------------------------------------------
## The format and Split will be done in the same step (this is to be done for files bigger than 1GB)
### The code will loop thru each chunk and will Format to Previse then will Split the file
### Each file will be a sensor
### If the sensor csv file has more than 1MM rows the code will split that file also
## -----------------------------------------------------------------------------------------

## The FormatToPrevise code is still Work in Progress for files bigger than 1 GB
### What to do
#### 1. Since the code does not load the whole CSV but splits it. The code needs to open each splitted file and do the Formatting
#### 2. The code will loop thru each Splitted CSV file and format each one of them
#### 3. Once formatted the code will split the file into 1MM rows per CSV

In [34]:
mdf = FormatToPrevise(df2_1,df2_2)

# Run following line to see the TOP 25 rows for the new Previse Format

In [37]:
mdf.head(25)

,DATETIME,TAGNAME,DESCRIPTION,VALUE
0,1/1/2014 0:00,BUH02EDPV1052,2K101 ESD VENT VALV [Open],0
1,1/1/2014 0:05,BUH02EDPV1052,2K101 ESD VENT VALV [Open],0
2,1/1/2014 0:10,BUH02EDPV1052,2K101 ESD VENT VALV [Open],0
3,1/1/2014 0:15,BUH02EDPV1052,2K101 ESD VENT VALV [Open],0
4,1/1/2014 0:20,BUH02EDPV1052,2K101 ESD VENT VALV [Open],0
5,1/1/2014 0:25,BUH02EDPV1052,2K101 ESD VENT VALV [Open],0
6,1/1/2014 0:30,BUH02EDPV1052,2K101 ESD VENT VALV [Open],0
7,1/1/2014 0:35,BUH02EDPV1052,2K101 ESD VENT VALV [Open],0
8,1/1/2014 0:40,BUH02EDPV1052,2K101 ESD VENT VALV [Open],0
9,1/1/2014 0:45,BUH02EDPV1052,2K101 ESD VENT VALV [Open],null


In [39]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105122 entries, 0 to 105121
Columns: 159 entries, DATETIME to BUH02PI1440
dtypes: object(159)
memory usage: 127.5+ MB


# Here we split the CSV data by file and each file will contain 1 million rows

### The SplitPreviseFormatCSVFile code is still Work in Progress for files bigger than 1 GB
### What to do

### Improvement
#### 1. I will update the code to create a CSV file per Sensor and if the sensor has more than 1MM rows it will splitted

In [50]:
SplitPreviseFormatCSVFile(mdf)

1
Exporting Tag_Data_2014_NoDups chunk1 Historian File

2
Exporting Tag_Data_2014_NoDups chunk2 Historian File

3
Exporting Tag_Data_2014_NoDups chunk3 Historian File

4
Exporting Tag_Data_2014_NoDups chunk4 Historian File

5
Exporting Tag_Data_2014_NoDups chunk5 Historian File

6
Exporting Tag_Data_2014_NoDups chunk6 Historian File

7
Exporting Tag_Data_2014_NoDups chunk7 Historian File

8
Exporting Tag_Data_2014_NoDups chunk8 Historian File

9
Exporting Tag_Data_2014_NoDups chunk9 Historian File

10
Exporting Tag_Data_2014_NoDups chunk10 Historian File

11
Exporting Tag_Data_2014_NoDups chunk11 Historian File

12
Exporting Tag_Data_2014_NoDups chunk12 Historian File

13
Exporting Tag_Data_2014_NoDups chunk13 Historian File

14
Exporting Tag_Data_2014_NoDups chunk14 Historian File

15
Exporting Tag_Data_2014_NoDups chunk15 Historian File

16
Exporting Tag_Data_2014_NoDups chunk16 Historian File

17
Exporting Tag_Data_2014_NoDups chunk17 Historian File

18
Exporting Tag_Data_2014_NoDup